In [ ]:
# -*- coding: utf-8
# @paidatocandeira
# Cria base com todos CNPJs e sócios do Brasil (Receita Federal) e mostra quais empresas são de deputados federais
#

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import unidecode

In [2]:
# Função para ler links de empresas nos Estados no site da Receita - retorna todas as linhas da página
def le_estado(url):
    r = requests.get(url)
    page_text = r.text
    lista_de_linha = page_text.split('\n')
    return lista_de_linha

In [3]:
# Função para retirar acentuação 
def f(str):
    return (unidecode.unidecode(str))

In [ ]:
# Lê página principal da Receita com dados das empresas, faz sopa e seleciona tabela com os links

In [3]:
res = requests.get("http://idg.receita.fazenda.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-abertos-do-cnpj")

In [4]:
soup =  BeautifulSoup(res.text, "lxml")

In [5]:
table = soup.select("table.plain")[0]

In [ ]:
# Listas que vão abrigar dados

In [6]:
empresas = []
socios = []

In [7]:
# Itera na tabela
for items in table.select("tr"):
    # Seleciona o atributo text - Estados - e o link - href - cria uma lista
    data = [[item.text,item['href']] for item in items.select(".external-link")]
    # Itera na lista
    for data_dict in data:
        # Chama função com o link href e retorna página inteira com dados do Estado
        pagina = le_estado(data_dict[1])
        # Guarda nome do Estado
        estado = data_dict[0]
        print(estado)
        # Itera na página do Estado
        for linha in pagina:
            # Segue uma série de condições de posições de informações, que estão no site principal da Receita para informações das empresas            
            if linha.startswith("01"):
                cod = linha[2:16]
                nome = linha[16:]
                #print(cod)
                #print(nome)
                # Abastece a lista com um dicionário das empresas de cada Estado
                empresas.append({'cnpj': str(cod), 'nome_empresa': str(nome).strip(), 'estado': str(estado)})
            elif linha.startswith("02"):
                cod2 = linha[2:16]
                indicador = linha[16:17]
                doc = linha[17:31]
                qualif = linha[31:33]
                socio = linha[33:]
                #print(cod2)
                #print(indicador)
                #print(doc)
                #print(qualif)
                #print(socio)
                # Abastece a lista com um dicionário dos sócios das empresas de cada Estado
                socios.append({'cnpj': str(cod2), 
                                'indicador': str(indicador), 
                                'documento': str(doc), 
                                'qualificacao': str(qualif), 
                                'socio': str(socio).strip(),
                                'estado': str(estado)})   

Acre
Espírito Santo
Paraíba
Rondônia
Alagoas
Goiás
Paraná
Roraima
Amapá
Maranhão
Pernambuco
Santa Catarina
Amazonas
Mato Grosso
Piauí
São Paulo
Bahia
Mato Grosso do Sul
Rio de Janeiro
Sergipe
Ceará
Minas Gerais
Rio Grande do Norte
Tocantins
Distrito Federal
Pará
Rio Grande do Sul


In [ ]:
# Cria dataframe com todas as empresas do Brasil

In [ ]:
df_empresas = pd.DataFrame(empresas, columns = ['cnpj', 'nome_empresa', 'estado'])

In [9]:
df_empresas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8952217 entries, 0 to 8952216
Data columns (total 3 columns):
cnpj            object
nome_empresa    object
estado          object
dtypes: object(3)
memory usage: 204.9+ MB


In [10]:
df_empresas.to_csv('empresas_br_4_1.csv', index=False)

In [ ]:
# Cria dataframe com os sócios de todas as empresas do Brasil

In [11]:
df_socios = pd.DataFrame(socios, columns = ['cnpj', 'indicador', 'documento', 'qualificacao', 'socio', 'estado'])

In [12]:
df_socios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17601862 entries, 0 to 17601861
Data columns (total 6 columns):
cnpj            object
indicador       object
documento       object
qualificacao    object
socio           object
estado          object
dtypes: object(6)
memory usage: 805.7+ MB


In [13]:
df_socios.to_csv('socios_br_4_1.csv', index=False)

In [7]:
# Acessa API da Câmara dos Deputados para pegar o link de API de cada deputado

In [6]:
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados'

In [7]:
deputados = []
for pagina in [1, 2, 3, 4, 5, 6]:
    parametros = {'formato': 'json', 'itens': 100, 'pagina': pagina}
    resposta = requests.get(url, parametros)
    for deputado in resposta.json()['dados']:
        dicionario = {"deputado": deputado['nome'], "link_api": deputado['uri']}
        deputados.append(dicionario)

In [8]:
# Cria dataframe com essa lista

In [9]:
df_deputados_api = pd.DataFrame(deputados)

In [10]:
# Itera no dataframe para vasculhar cada API dos deputados e pegar os dados de cada um

In [11]:
perfis = []
for num, row in df_deputados_api.iterrows():
    parametros = {'formato': 'json'}
    resposta = requests.get(row['link_api'], parametros)
    linha = resposta.json()['dados']
    api = linha['uri']
    nomecompleto = linha['nomeCivil'].upper()
    sexo = linha['sexo']
    nascimento = linha['dataNascimento']
    linha2 = resposta.json()['dados']['ultimoStatus']
    nomeurna = linha2['nomeEleitoral']
    partido = linha2['siglaPartido']
    uf = linha2['siglaUf']
    foto = linha2['urlFoto']
    linha3 = resposta.json()['dados']['ultimoStatus']['gabinete']
    tel = linha3['telefone']
    e_mail = linha3['email']
    dicionario = {"link_api": api, "nome_completo": nomecompleto, "nome_eleitoral": nomeurna, "partido": partido, "uf": uf, "link_foto": foto, "telefone": tel, "e_mail": e_mail, "sexo": sexo, "data_nascimento": nascimento}
    perfis.append(dicionario)

In [12]:
# Cria dataframe final com os 513 deputados

In [13]:
perfis_deputados = pd.DataFrame(perfis)

In [14]:
# Retira acentuação da coluna

In [15]:
perfis_deputados["nome_completo"] = perfis_deputados["nome_completo"].apply(f)

In [16]:
perfis_deputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 10 columns):
data_nascimento    513 non-null object
e_mail             513 non-null object
link_api           513 non-null object
link_foto          513 non-null object
nome_completo      513 non-null object
nome_eleitoral     513 non-null object
partido            513 non-null object
sexo               513 non-null object
telefone           512 non-null object
uf                 513 non-null object
dtypes: object(10)
memory usage: 40.2+ KB


In [17]:
perfis_deputados.to_csv('perfis_deputados_5_1.csv', index=False)

In [ ]:
# Cria dataframe a partir dos arquivos de empresas e sócios

In [4]:
socios_brasil = pd.read_csv("socios_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [5]:
socios_brasil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17601862 entries, 0 to 17601861
Data columns (total 7 columns):
Unnamed: 0      int64
cnpj            object
indicador       int64
documento       object
qualificacao    int64
socio           object
estado          object
dtypes: int64(3), object(4)
memory usage: 940.0+ MB


In [18]:
socios_brasil.reset_index().tail()

,index,Unnamed: 0,cnpj,indicador,documento,qualificacao,socio,estado
17601857,17601857,17601857,29236068000156,2,,65,CRISTINA DE MEDEIROS HUISMAN,Rio Grande do Sul
17601858,17601858,17601858,29236075000158,2,,65,CLAUDIOMIRO REIS,Rio Grande do Sul
17601859,17601859,17601859,29236906000191,2,,16,RONALDO SOUZA DE OLIVEIRA,Rio Grande do Sul
17601860,17601860,17601860,29237812000137,2,,49,ADELAR ANTONIO MORARI,Rio Grande do Sul
17601861,17601861,17601861,29237812000137,2,,22,JEFERSON MARASCA MORARI,Rio Grande do Sul


In [19]:
# Cruzamento dos nomes completos dos deputados com os nomes de sócios

In [20]:
deputados_socios = pd.merge(perfis_deputados, socios_brasil, left_on='nome_completo', right_on='socio')

In [21]:
deputados_socios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2086 entries, 0 to 2085
Data columns (total 17 columns):
data_nascimento    2086 non-null object
e_mail             2086 non-null object
link_api           2086 non-null object
link_foto          2086 non-null object
nome_completo      2086 non-null object
nome_eleitoral     2086 non-null object
partido            2086 non-null object
sexo               2086 non-null object
telefone           2083 non-null object
uf                 2086 non-null object
Unnamed: 0         2086 non-null int64
cnpj               2086 non-null object
indicador          2086 non-null int64
documento          2086 non-null object
qualificacao       2086 non-null int64
socio              2086 non-null object
estado             2086 non-null object
dtypes: int64(3), object(14)
memory usage: 293.3+ KB


In [ ]:
# Carrega nomes de empresas

In [22]:
empresas_brasil = pd.read_csv("empresas_br_4_1.csv",sep=',',encoding = 'utf-8', converters={'cnpj': lambda x: str(x)})

In [23]:
empresas_brasil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8952217 entries, 0 to 8952216
Data columns (total 4 columns):
Unnamed: 0      int64
cnpj            object
nome_empresa    object
estado          object
dtypes: int64(1), object(3)
memory usage: 273.2+ MB


In [24]:
# Mostra os nomes das empresas dos deputados - a partir do dataframe criado acima

In [25]:
deputados_socios_empresas = pd.merge(deputados_socios, empresas_brasil, left_on='cnpj', right_on='cnpj')

In [26]:
deputados_socios_empresas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2086 entries, 0 to 2085
Data columns (total 20 columns):
data_nascimento    2086 non-null object
e_mail             2086 non-null object
link_api           2086 non-null object
link_foto          2086 non-null object
nome_completo      2086 non-null object
nome_eleitoral     2086 non-null object
partido            2086 non-null object
sexo               2086 non-null object
telefone           2083 non-null object
uf                 2086 non-null object
Unnamed: 0_x       2086 non-null int64
cnpj               2086 non-null object
indicador          2086 non-null int64
documento          2086 non-null object
qualificacao       2086 non-null int64
socio              2086 non-null object
estado_x           2086 non-null object
Unnamed: 0_y       2086 non-null int64
nome_empresa       2086 non-null object
estado_y           2086 non-null object
dtypes: int64(4), object(16)
memory usage: 342.2+ KB


In [27]:
deputados_socios_empresas.to_csv('empresas_deputados_5_1.csv', index=False)

In [28]:
deputados_socios_empresas.reset_index().head()

,index,data_nascimento,e_mail,link_api,link_foto,nome_completo,nome_eleitoral,partido,sexo,telefone,...,Unnamed: 0_x,cnpj,indicador,documento,qualificacao,socio,estado_x,Unnamed: 0_y,nome_empresa,estado_y
0,0,1962-03-29,dep.abelmesquitajr.@camara.leg.br,https://dadosabertos.camara.leg.br/api/v2/depu...,http://www.camara.leg.br/internet/deputado/ban...,ABEL SALVADOR MESQUITA JUNIOR,ABEL MESQUITA JR.,DEM,M,3215-5248,...,2752659,00376437000124,2,,22,ABEL SALVADOR MESQUITA JUNIOR,Roraima,1434026,AUTO POSTO ABEL GALINHA LIMITADA,Roraima
1,1,1962-03-29,dep.abelmesquitajr.@camara.leg.br,https://dadosabertos.camara.leg.br/api/v2/depu...,http://www.camara.leg.br/internet/deputado/ban...,ABEL SALVADOR MESQUITA JUNIOR,ABEL MESQUITA JR.,DEM,M,3215-5248,...,2757584,04651634000165,2,,22,ABEL SALVADOR MESQUITA JUNIOR,Roraima,1436671,SALOMAO VEICULO LTDA,Roraima
2,2,1963-07-11,dep.adailcarneiro@camara.leg.br,https://dadosabertos.camara.leg.br/api/v2/depu...,http://www.camara.leg.br/internet/deputado/ban...,JOSE ADAIL CARNEIRO SILVA,ADAIL CARNEIRO,PP,M,3215-5335,...,13100369,03313310000154,2,,49,JOSE ADAIL CARNEIRO SILVA,Ceará,6639984,MOVEMENT FOMENTO COMERCIAL LTDA,Ceará
3,3,1963-07-11,dep.adailcarneiro@camara.leg.br,https://dadosabertos.camara.leg.br/api/v2/depu...,http://www.camara.leg.br/internet/deputado/ban...,JOSE ADAIL CARNEIRO SILVA,ADAIL CARNEIRO,PP,M,3215-5335,...,13177060,04457059000164,2,,49,JOSE ADAIL CARNEIRO SILVA,Ceará,6684193,MOVEMENT CORRETAGEM & COMERCIO DE VEICULOS LTDA,Ceará
4,4,1958-02-10,dep.adalbertocavalcanti@camara.leg.br,https://dadosabertos.camara.leg.br/api/v2/depu...,http://www.camara.leg.br/internet/deputado/ban...,ADALBERTO CAVALCANTI RODRIGUES,ADALBERTO CAVALCANTI,AVANTE,M,3215-5402,...,3018287,03483627000139,2,,49,ADALBERTO CAVALCANTI RODRIGUES,Pernambuco,1592684,LUCIA MARIANO & CIA LTDA - ME,Pernambuco


In [29]:
# Bônus - procura por nome individual

In [30]:
nome = socios_brasil[(socios_brasil['socio'] == 'MICHEL MIGUEL ELIAS TEMER LULIA')]

In [31]:
nome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 5169384 to 8772429
Data columns (total 7 columns):
Unnamed: 0      3 non-null int64
cnpj            3 non-null object
indicador       3 non-null int64
documento       3 non-null object
qualificacao    3 non-null int64
socio           3 non-null object
estado          3 non-null object
dtypes: int64(3), object(4)
memory usage: 192.0+ bytes


In [32]:
nome_socios = pd.merge(nome, empresas_brasil, left_on='cnpj', right_on='cnpj')

In [33]:
nome_socios.reset_index()

,index,Unnamed: 0_x,cnpj,indicador,documento,qualificacao,socio,estado_x,Unnamed: 0_y,nome_empresa,estado_y
0,0,5169384,05397402000195,2,,28,MICHEL MIGUEL ELIAS TEMER LULIA,São Paulo,2701368,TEMER ADVOGADOS ASSOCIADOS,São Paulo
1,1,6566575,59492884000191,2,,16,MICHEL MIGUEL ELIAS TEMER LULIA,São Paulo,3390454,CENTRO DE ESTUDOS CONSTITUCIONAIS=CEC,São Paulo
2,2,8772429,12982441000115,2,,22,MICHEL MIGUEL ELIAS TEMER LULIA,São Paulo,4434037,TABAPUA INVESTIMENTOS E PARTICIPACOES LTDA,São Paulo
